In [ ]:
import pandas as pd
import json
import plotly.express as px

In [ ]:
with open("/GeneGraphDB/data/20220308_neo4j_colocalization/cas1.json") as f:
    cas1_dict = json.load(f)

In [ ]:
cas1_df = pd.DataFrame(cas1_dict.values())
print(len(cas1_df), "rows total")
cas1_df = cas1_df[cas1_df['num_tgt_p100s'] > 1]
print(len(cas1_df), "filtered for >1 tgt p100")
cas1_df = cas1_df[cas1_df['num_colocated_p100s'] > 1]
print(len(cas1_df), "+ filtered for >1 shared edge")

cas1_df = cas1_df[cas1_df['tgt_colocalization'] > 0.1]
print(len(cas1_df), "+ filtered for >0.1 tgt colocalization")

cas1_df = cas1_df[cas1_df['bait_colocalization'] > 0.01]
print(len(cas1_df), "+ filtered for >0.01 bait colocalization")


cas1_df.sort_values('tgt_colocalization', ascending=False)

In [ ]:
sum(cas1_df["num_tgt_p100s"] >= 20)

In [ ]:
fig = px.histogram(cas1_df[cas1_df["num_tgt_p100s"] >= 5], x="num_tgt_p100s")
fig.show()

In [ ]:
cas1_df = cas1_df[cas1_df["num_tgt_p100s"] >= 5]